In [1]:
import json
import xml.etree.ElementTree as ET
import os
import glob
import itertools
from time import sleep
from pprint import pprint
from tqdm import tqdm
import pandas as pd

In [2]:
import requests
SLEEP_SECONDS = 0.011
offset = 0
limit = 1000
headers = {"x-api-key": "5NNyAuuUBm4VAUZiEQOL85IeBVtNf9p3tpEPleoe"}

In [3]:
import re
code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')
def cleaning(l:list):
    if (len(l) == 0):
        return set()
    else:
        l = [code_regex.sub('',x) for x in l]
        l = [x.lower() for x in l]
        return set(l)


In [4]:
paths = glob.glob("/home/sibava/survey/dataset/grobid_full_text/*")

In [4]:
xml_errs = ("[BAD_INPUT_DATA]","[NO_BLOCKS]","[TOO_MANY_TOKENS]","[TIMEOUT]","[GENERAL]")
ns = {"ns":"http://www.tei-c.org/ns/1.0"}

In [4]:
aan_df = pd.read_json("/home/sibava/moonshot/selected_survey.jsonl",lines=True)

In [5]:
aan_df = aan_df.query("year > 2014")
aan_titles = cleaning(aan_df['title'])

In [ ]:
survey_paths = []
for path in tqdm(paths):
    with open(path) as f:
        if(f.read().startswith(xml_errs)):
            # print("err")
            continue
    tree = ET.parse(path)
    root = tree.find("{http://www.tei-c.org/ns/1.0}teiHeader")
    for c in root.iter("{http://www.tei-c.org/ns/1.0}title"):
        if c.text is  not None and "survey" in c.text.lower() and code_regex.sub('',c.text).lower() in aan_titles:
            survey_paths.append(path)
            print(path)

In [90]:
pd.DataFrame(list(set(survey_paths)),columns=["path"]).to_csv("/home/sibava/survey/dataset/survey_path.csv",index=False)

In [6]:
surveys_paths = pd.read_csv("/home/sibava/survey/dataset/survey_path.csv")["path"].to_list()

In [7]:
c = 0
title_ref = {}
for path in surveys_paths:
    tree = ET.parse(path)
    root = tree.getroot()
    title = root.find("ns:teiHeader/ns:fileDesc/ns:titleStmt/ns:title",ns).text
    refs = []
    for bib in root.iter("{http://www.tei-c.org/ns/1.0}biblStruct"):
        for t in bib.findall("ns:analytic/ns:title",ns):
            if t.text is not None:
                refs.append(t.text)
    title_ref[title] = refs
    # print(len(refs))

In [8]:
aan_df['ref_titles'] = aan_df['references'].map(lambda l:[x["title"] for x in l])

In [16]:
title_ref_id = []
for index,row in aan_df.iterrows():
    survey_title = row["title"]
    paper_id = row["paperId"]
    aan_ref = row['ref_titles']
    if(survey_title in title_ref):
        aac_ref = title_ref[survey_title]
    else:
        aac_ref = []
    refs = cleaning(aan_ref)| cleaning(aac_ref)
    title_ref_id.append({"title":survey_title,"paperId":paper_id,"refs":refs})

In [21]:
df = pd.DataFrame(title_ref_id)

In [75]:
refs_ids =[]
for index,row in tqdm(df.iterrows()):
    refs = row['refs']
    ref_ids = []
    for ref in refs:
        responce = requests.get(
            f"http://api.semanticscholar.org/graph/v1/paper/search?query={ref}&offset={offset}&limit={limit}&fields=title,venue,fieldsOfStudy",
            headers=headers
        )
        txt = responce.text
        d = json.loads(txt)
        for r in d['data']:
            if code_regex.sub('',r['title']).lower() == ref:
                ref_ids.append(r['paperId'])
    refs_ids.append(ref_ids)
# df['refs_ids'] = refs_ids

75it [1:33:03, 74.45s/it] 


In [77]:
df['refs_ids'] = refs_ids

In [85]:
df['refs_ids'].map(lambda x:len(x))

0      58
1      45
2     197
3      59
4      65
     ... 
70      7
71      4
72     22
73     23
74     36
Name: refs_ids, Length: 75, dtype: int64

In [88]:
df.to_json("/home/sibava/survey/dataset/survey_ref.jsonl",orient="records",lines=True)

In [5]:
df = pd.read_json("/home/sibava/survey/dataset/survey_ref.jsonl",orient="records",lines=True)

In [11]:
df['refs'].map(lambda x:len(x)).mean()

91.22666666666667

In [6]:
new_df = df[df['refs_ids'].map(lambda x:len(x) >10 )]

In [7]:
new_df.columns

Index(['title', 'paperId', 'refs', 'refs_ids'], dtype='object')

In [22]:
embeddings = []
referenceCounts = []
years = []

for index,row in tqdm(new_df.iterrows()):
    responce = requests.get(
        f"http://api.semanticscholar.org/graph/v1/paper/{row['paperId']}?fields=title,year,referenceCount,embedding",
        headers=headers,
    )
    txt = responce.text
    d = json.loads(txt)
    embeddings.append(d['embedding'])
    years.append(d['year'])
    referenceCounts.append(d['referenceCount'])


70it [00:30,  2.33it/s]


In [23]:
new_df['embedding'] = embeddings
new_df['year'] = years
new_df['referenceCount'] = referenceCounts

/tmp/ipykernel_1662659/924296600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['embedding'] = embeddings
/tmp/ipykernel_1662659/924296600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['year'] = years
/tmp/ipykernel_1662659/924296600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [30]:
new_df = new_df.drop('citationCount',axis=1)

In [33]:
new_df.to_json("/home/sibava/survey/dataset/training.jsonl",orient="records",lines=True)

In [13]:
reference_ids = list(set(itertools.chain.from_iterable(new_df['refs_ids'])))
references = []

for ref_id in tqdm(reference_ids):
    responce = requests.get(
        f"http://api.semanticscholar.org/graph/v1/paper/{ref_id}?offset={offset}&limit={limit}&fields=title,embedding",
        headers=headers,
        )
    txt = responce.text
    d = json.loads(txt)
    references.append(d)
    sleep(SLEEP_SECONDS)

100%|██████████| 4185/4185 [40:45<00:00,  1.71it/s]  


In [15]:
ref_df = pd.DataFrame(references)

In [19]:
ref_df.to_json("/home/sibava/survey/dataset/references.jsonl",orient="records",lines=True)